In [1]:
import gym
import gym_simpleflappy
from tqdm import tqdm
import neat
import numpy as np
from matplotlib import pyplot as plt
import random

env = gym.make("SimpleFlappyDistance-v0")
env.reset()

max_eps = 1000
episode = 0
steps = 0
screen_width = 288
screen_height = 512
max_speed = 10.0


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\users\sofik\appdata\local\programs\python\python37\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Environment '<class 'gym_simpleflappy.envs.flappy_env.FlappyEnvDistance'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
observation = env.reset()
# [bird height, bird speed, distance to next pipe, height of next pipe, distance to second pipe, height of second pipe]

In [3]:
xdim = 130
ydim = 130
vdim = 20

q_table = np.zeros((xdim, ydim, vdim, 2))
q_table

array([[[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        ...,

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
   

In [4]:
learning_rate = .7
discount_factor = 0.95
max_speed = 10.0


In [5]:
# horizontal distance of the bird from the pipe
# vertical distance of the bird from the pipe
# vertical speed of the bird

# [bird height, bird speed, distance to next pipe, height of next pipe, distance to second pipe, height of second pipe]


def preprocess_features(feats):
    # features => [bird speed, hor. distance to next pipe, ver. distance to next pipe]
    new_feats = [feats[1], feats[2], feats[3] - feats[0]]
#     new_feats[0] /= max_speed
#     new_feats[1] /= screen_width
#     new_feats[2] /= screen_height    
    return new_feats

In [6]:
done = False
steps = 0
epsilon = 0.1
ground_y = screen_height * 0.9
pipe_max = ground_y - screen_height * 0.1
pipe_min = screen_height * 0.4

observation = env.reset()
v,x,y = preprocess_features(list(observation))
print(v,x,y)

while not done:  #  Game cycle            
    if random.random() <= epsilon:
        action = random.randrange(2)
    elif q_table[x,y,v,0] >= q_table[x,y,v,1]:
        action = 0
    else:
        action = 1
        
    observation, reward, done, info = env.step(action)  #  Make a move
    v1, x1, y1 = preprocess_features(list(observation))

    steps += 1
    abs_vel = abs(v) + (max_speed if v > 0 else 0)
    abs_vel1 = abs(v1) + (max_speed if v > 0 else 0)
    q_table[x,y,abs_vel,action] = (1 - learning_rate) * (q_table[x,y,abs_vel,action]) + learning_rate * ( 
        reward + (discount_factor)*max(q_table[x1,y1,abs_vel1, 0],
                                       q_table[x1,y1,abs_vel1,1]))
    x,y,z = x1,y1,z1
    

0 172.8 -24.94691314668887


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
ground_y = screen_height * 0.9
pipe_max = ground_y - screen_height * 0.1
pipe_min = screen_height * 0.4
pipe_min